In [1]:
!nvidia-smi

Tue Apr 15 23:50:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.83                 Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   63C    P8              6W /   80W |     488MiB /   6144MiB |     29%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
data_dir = 'Product_Classification'  

In [3]:
import os
import csv
from  datetime import datetime
from ultralytics import YOLO
import cv2
import numpy as np

def train_model():
    # Initialize YOLO model
    model= YOLO('yolov8n.pt')
    
    # Train the model
    results = model.train(
        data=r'C:\Users\gifly\Desktop\DataScience\Project\Product_Classification\data.yaml',
        epochs=50,
        batch=16,
        imgsz=640,
        device=0,
        patience=5,
    )
    
    return model

def initialize_csv():
    # Create or update CSV file with initial counts
    csv_path = 'object_counts.csv'
    if not os.path.exists(csv_path):
        with open(csv_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Object', 'ID', 'NUM'])
            writer.writerow(['Dryer', 'D0001', '0'])
            writer.writerow(['IRON', 'I0002', '0'])
    return csv_path

def update_csv_counts(new_counts):
    csv_path = 'object_counts.csv'
    
    # Check if the CSV file exists
    if os.path.exists(csv_path):
        # Read the existing CSV file
        with open(csv_path, 'r') as file:
            reader = csv.DictReader(file)
            rows = list(reader)
        
        # Update the counts
        for row in rows:
            if row['Object'] == 'Dryer':
                row['NUM'] = str(int(row['NUM']) + new_counts['Dryer'])
            elif row['Object'] == 'IRON':
                row['NUM'] = str(int(row['NUM']) + new_counts['IRON'])
        
        # Write the updated counts back to the CSV file
        with open(csv_path, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['Object', 'ID', 'NUM'])
            writer.writeheader()
            writer.writerows(rows)
    else:
        # Create a new CSV file if it doesn't exist
        with open(csv_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Object', 'ID', 'NUM'])
            writer.writerow(['Dryer', 'D0001', str(new_counts['Dryer'])])
            writer.writerow(['IRON', 'I0002', str(new_counts['IRON'])])

def detect_and_count(model, image_path):
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image {image_path}")
        return None
    
    # Perform detection
    results = model(image)
    
    # Initialize counts
    counts = {'Dryer': 0, 'IRON': 0}
    
    # Process detections
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Get class name
            class_id = int(box.cls[0])
            class_name = model.names[class_id]
            
            # Increment count
            counts[class_name] += 1
            
            # Draw bounding box and label
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            
            # Draw rectangle
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Add label
            label = f"{class_name} {conf:.2f}"
            cv2.putText(image, label, (x1, y1 - 10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Add total counts to image
    count_text = f"Dryer: {counts['Dryer']}, IRON: {counts['IRON']}"
    cv2.putText(image, count_text, (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Save annotated image
    output_path = f"detected_{os.path.basename(image_path)}"
    cv2.imwrite(output_path, image)
    
    return counts

def main():
    # Step 1: Train the model
    print("Training model...")
    model = train_model()
    
    # Step 2: Initialize CSV
    csv_path = initialize_csv()
    print(f"CSV file initialized at {csv_path}")
    
    # Step 3: Process test images
    test_dir = "test/images"
    if os.path.exists(test_dir):
        for image_file in os.listdir(test_dir):
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(test_dir, image_file)
                print(f"\nProcessing {image_path}")
                
                # Detect and count objects
                counts = detect_and_count(model, image_path)
                if counts:
                    # Update CSV with new counts
                    update_csv_counts(counts)
                    print(f"Updated counts: Dryer={counts['Dryer']}, IRON={counts['IRON']}")
    
    print("\nProcessing complete!")


if __name__ == "__main__":
    main()


Training model...
Ultralytics 8.3.108  Python-3.11.2 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\gifly\Desktop\DataScience\Project\Product_Classification\data.yaml, epochs=50, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

train: Scanning C:\Users\gifly\Desktop\DataScience\Project\Product_Classification\train\labels.cache... 759 images, 9 backgrounds, 0 corrupt: 100%|██████████| 759/759 [00:00<?, ?it/s]
val: Scanning C:\Users\gifly\Desktop\DataScience\Project\Product_Classification\valid\labels.cache... 70 images, 1 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]


Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      4.98G     0.5342      1.759     0.9806         21        640: 100%|██████████| 48/48 [00:19<00:00,  2.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.07it/s]

                   all         70         75      0.789      0.723      0.834       0.71


...

      15/50      5.02G     0.3767     0.5371     0.8906         18        640: 100%|██████████| 48/48 [00:17<00:00,  2.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.13it/s]

                   all         70         75      0.815      0.928      0.952      0.927
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 10, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



15 epochs completed in 0.078 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 6.2MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics 8.3.108  Python-3.11.2 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.58it/s]


                   all         70         75      0.772      0.911      0.947      0.932
                 Dryer         29         29      0.638      0.931      0.928       0.91
                  IRON         46         46      0.905      0.891      0.966      0.955
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs\detect\train3
CSV file initialized at object_counts.csv

Processing complete!


In [4]:
# import os
# import csv
# from datetime import datetime
# from ultralytics import YOLO
# import cv2
# import numpy as np

# def train_model():
#     # Initialize YOLO model
#     model = YOLO('yolov8n.pt')
    
#     # Train the model
#     results = model.train(
#         data=r'C:\Users\gifly\Desktop\DataScience\Project\Product_Classification\data.yaml',
#         epochs=5,
#         batch=16,
#         imgsz=640,
#         device=0
#     )
    
#     return model

# def initialize_csv():
#     # Create or update CSV file with initial counts
#     csv_path = 'object_counts.csv'
#     if not os.path.exists(csv_path):
#         with open(csv_path, 'w', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow(['Object', 'ID', 'NUM'])
#             writer.writerow(['Dryer', 'D0001', '0'])
#             writer.writerow(['IRON', 'I0002', '0'])
#     return csv_path

# def update_csv_counts(counts):
#     csv_path = 'object_counts.csv'
#     with open(csv_path, 'w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Object', 'ID', 'NUM'])
#         writer.writerow(['Dryer', 'D0001', str(counts['Dryer'])])
#         writer.writerow(['IRON', 'I0002', str(counts['IRON'])])

# def detect_and_count(model, image_path):
#     # Read image
#     image = cv2.imread(image_path)
#     if image is None:
#         print(f"Error: Could not read image {image_path}")
#         return None
    
#     # Perform detection
#     results = model(image)
    
#     # Initialize counts
#     counts = {'Dryer': 0, 'IRON': 0}
    
#     # Process detections
#     for result in results:
#         boxes = result.boxes
#         for box in boxes:
#             # Get class name
#             class_id = int(box.cls[0])
#             class_name = model.names[class_id]
            
#             # Increment count
#             counts[class_name] += 1
            
#             # Draw bounding box and label
#             x1, y1, x2, y2 = map(int, box.xyxy[0])
#             conf = float(box.conf[0])
            
#             # Draw rectangle
#             cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
#             # Add label
#             label = f"{class_name} {conf:.2f}"
#             cv2.putText(image, label, (x1, y1 - 10), 
#                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
#     # Add total counts to image
#     count_text = f"Dryer: {counts['Dryer']}, IRON: {counts['IRON']}"
#     cv2.putText(image, count_text, (10, 30), 
#                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
#     # Save annotated image
#     output_path = f"detected_{os.path.basename(image_path)}"
#     cv2.imwrite(output_path, image)
    
#     return counts

# def main():
#     # Step 1: Train the model
#     print("Training model...")
#     model = train_model()
    
#     # Step 2: Initialize CSV
#     csv_path = initialize_csv()
#     print(f"CSV file initialized at {csv_path}")
    
#     # Step 3: Process test images
#     test_dir = "test/images"
#     if os.path.exists(test_dir):
#         for image_file in os.listdir(test_dir):
#             if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
#                 image_path = os.path.join(test_dir, image_file)
#                 print(f"\nProcessing {image_path}")
                
#                 # Detect and count objects
#                 counts = detect_and_count(model, image_path)
#                 if counts:
#                     # Update CSV with new counts
#                     update_csv_counts(counts)
#                     print(f"Updated counts: Dryer={counts['Dryer']}, IRON={counts['IRON']}")
    
#     print("\nProcessing complete!")

# if __name__ == "__main__":
#     main()